# Word2Vec Models for NLP

### The Word2Vec model is a simple word embedding neural network, developed by Mikolov et al. (2013).
####  *Such continuous word embedding representations have have been proven to be able to carry semantic meanings and are useful in various NLP tasks.*

## Motivation

In this notebook, I have attempted to implement three language models described in *Le & Mikolov (2014)*'s paper ** *Distributed Representations of Sentences and Documents* **.

The implementations don't make use of any NLP libraries and consist of the simplest form of the algorithm with little optimization.
The aim of this notebook is simply to gain:
* understanding of the **language models' algorithm**
* intuition on **word embedding representations**
* understanding of inner workings of **neural networks**

## Introduction

In NLP, popular fixed-length features used in language models are **bag-of-words**.

However bag-of-words features have two major weaknesses: 
- they lose the **ordering** of the words.
- they also ignore **semantics** of the words. 

In the paper by **Le & Mikolov (2014)**, they propose a distributed representation of sentences and documents, which they call ** *Paragraph Vector* **. 

The model assumes the ** *Distributional Hypothesis* ** that words are characterized by other words in their vicinity. This idea is used to estimate the probability of two words occurring near each other.

*Paragraph Vector* is an **unsupervised algorithm** that learns fixed-length feature representations from **variable-length pieces of texts**, such as sentences and paragraphs. The algorithm represents each word and sentence by a dense vector which is trained to predict words in the document. Its construction gives the algorithm the potential to overcome the **weaknesses** of bag-of- words models. 

<img src="image/3.png"  width="500" height="200">

Empirical results, in the paper, show that **Paragraph Vectors outperform bag-of-words models**, as well as, other techniques for text representations. For example, **word weighting functions** (which requires task-specific tuning) and **parse trees**. The authors were able to achieve new **state-of-the-art** results on several **text classification** and **sentiment analysis** tasks.

This is made possible because words with **similar meaning** are mapped to a **similar position** in the **vector space**. The difference between word vectors also carry meaning. For example, the word vectors can be used to answer analogy questions using simple vector algebra: “King” - “man” + “woman” = “Queen”.
Theses properties of word vectors are useful in many NLP task, such as **language modeling and understanding** and **machine translation.**

## One-Word Context Language Model

For this  first language model, we will look at the simplest form of the *Continuous Bag-Of-Word Model (CBOW)* introduced in *Mikolov et al. (2013a)*. This model has just **one word** as the context to predict the next target word in the sentence. This context vector is fed into a **neural network** with a **single hidden layer** that is fully connected.

![](image/1neural-network.png)

The input vector is a **one-hot encoded vector**. Each word is mapped into its own **unique vector of features**, represented by a column in the matrix $W_I$. There is also another weight matrix, $W_O$ connecting the hidden layer to the output layer.

Given a **context word** and these **weight matrices**, we can compute a **score** for each word in the vocabulary. This is essentially a **multiclass classification** where we have as many labels as our vocabulary size $V$. **Softmax regression**, a log-linear classification model, is used to compute the **posterior probability $P(W_O|W_I)$:**

$$ P(W_O|W_I) = y_i = \frac{exp(W_I \cdot W'^T_O)}{\sum^V_{j=1} exp(W_I \cdot W'^T_j)} $$

In the paper, **hierarchical softmax** is used for faster training, but for simpliticy I will just use a regular softmax regression as the multiclass classifier.

### Initialization

The continuous embeddings of the words represent **points in a vector space**. What the Word2Vec model is trying to do is assign each word a **meaningful point** in that space that represent its **semantic** in **relation** to the other words. The words are first **initialized to a random location** in that space, the model will then try  to learn better vector positions. Similar words will be **pushed together**, while different ones will be **pulled away.** 

### Updating the hidden-to-output layer weights

The model learns by maximzing the posterior probability. For **numerical stability**, we will instead **minimize the log loss function**: 

$$E = -\log P(W_O|W_I)$$

$$E = -\log \frac{exp(W_I \cdot W'^T_O)}{\sum^V_{j=1} exp(W_I \cdot W'^T_j)} =-\log \frac{exp(u)}{\sum^V_{j=1} exp(u_j)} $$

$$E = -u + \log \sum^V_{j=1} exp(u_j)$$

The update equation of the weights between **hidden and output layers**:

$$\dfrac{\partial E}{\partial u} = -t_j + \frac{exp(u)}{\sum^V_{j=1} exp(u_j)}$$
$$ \dfrac{\partial E}{\partial u} = P(W_O|W_I) - t_j = e_j$$

Where $t_j = 1$ if the **output word is the actual target**, otherwise $t_j = 0$. This derivative is also the **prediction error of the output layer**, $e_j$.

The **gradient** on the hidden-to-output weights is:
$$ \dfrac{\partial E}{\partial W_O} = \dfrac{\partial E}{\partial u_j} \cdot \dfrac{\partial u_j}{\partial W_O} = e_j \cdot h_i$$

Where $h_i$ is a copy of the vector corresponding to the **input word**. 

To **update the  output weights**, $W_O$, for the hidden-to-output layer, **stochastic gradient descent** is used with a learning rate $\alpha$: 

$${W_{O}}^{T (new)}_j = {W_{O}}^{T (old)}_j - \alpha \cdot e_j \cdot h_j$$

### Updating the input-to-hidden layer weights

The **prediction errors** need to be **backpropagated** to the input-to-hidden weights. 

The derivative of $E$ on the output of the hidden layer is:

$$ \dfrac{\partial E}{\partial h_i} = \sum^V_{j=1} \dfrac{\partial E}{\partial u_j} \cdot \dfrac{\partial u_j}{\partial h_i} = \sum^V_{j=1} e_j \cdot W_O = EH$$

EH is the **sum of the hidden-to-output vectors** for each word in the vocabulary **weighted** by their **prediction error**.

The **gradient** on the input weights is:

$$ \dfrac{\partial E}{\partial W_I} = \dfrac{\partial E}{\partial h_i} \cdot \dfrac{\partial h_i}{\partial W_I} = EH_i \cdot x$$

Where $x$ is one-hot encoded vector.

To **update the  intput weights** for the input-to-hidden layer, we make use again of **stochastic gradient descent**, with a learning rate $\alpha$: 

$${W_{I}}^{T (new)}_j = {W_{I}}^{T (old)}_j - \alpha \cdot EH $$

### Python Implementation

Now that we have good idea how the model works, we'd like to see it in action. To keep this notebook **concise and readable** I have kept the code in separate files located in the same directory of this notebook.

This **first model** can be found in file * "Word2Vec_1WordContext.py" *

Let's test this model with a *tiny* toy dataset: 

In [1]:
sentences = ['<s> the prince loves skateboarding in the park </s>', 
             '<s> the princess loves the prince but the princess hates skateboarding </s>',
             '<s> skateboarding in the park is popular </s>',
             '<s> the prince is popular but the prince hates attention </s>',
             '<s> the princess loves attention but the princess hates the park </s>']

We should not expect to learn much with such a small sample. Furthermore, most **neural networks** perform very poorly on small datasets. However, the goal is to test if the model was implemented correctly and gain some **intuition** on how word vectors representation works. 

Tokens $<s>$  and $</s>$ have been added at the beginning and end of sentences respectively. This conveniently allows the context window to loop over every word **equally** (more on that later).

Let's import the file and **instantiate the model** with our tiny dataset with **three nodes** in the hidden layer and learning rate of 1.2.

In [2]:
from Word2Vec_1WordContext import Word2Vec_1WordContext

model1 = Word2Vec_1WordContext(sentences, learning_rate = 1.2, nodes_HL = 3)

The model is kept simple with only **three hidden nodes** because with such little data not much can be learned. In fact, we are actually better off constraining the model to **low dimensionality** in that case. In addition, three nodes will allow us to plot our word vectors on a **3D graph** and **visualize the neural network features**.

The training process has been simplified to a **single iteration for every word in the text**. The function returns the learned **input weight matrix** containing the **word vectors**, and a dictionary of the **text's vocabulary** where the values are the corresponding **row indices** of the weight matrix.

In [5]:
W, vocab = model1.train()

print( "Vocabulary :")
for kv in vocab.items():
    print( kv)
print( "\n Learned Word Vectors: \n", W)

Vocabulary :
('<s>', 0)
('the', 1)
('prince', 2)
('loves', 3)
('skateboarding', 4)
('in', 5)
('park', 6)
('</s>', 7)
('princess', 8)
('but', 9)
('hates', 10)
('is', 11)
('popular', 12)
('attention', 13)

 Learned Word Vectors: 
 [[-2.08270488  2.55925557 -2.12930501]
 [-6.60210368  8.58476006 -6.80716901]
 [-1.95048465  2.54917797 -1.94794019]
 [-1.90633806  2.23921986 -1.77413093]
 [-0.62354432  0.42858976 -0.31709678]
 [-0.17637633  0.21253038 -0.08490952]
 [-1.80168588  2.38031994 -1.66203477]
 [-0.10635291 -0.09666614  0.13161846]
 [-3.29223587  4.05530739 -3.08323329]
 [-2.05342986  2.44019742 -1.84177854]
 [-2.74530684  3.82105132 -2.88493793]
 [-0.40716979  0.79752959 -0.37286327]
 [-0.65052351  0.69169371 -0.3817471 ]
 [-2.86758368  3.64153825 -2.80406576]]


It **worked!** You maybe trust me, but you haven't gained any **intuition** on what has happened. That's the **problem** with **neural nets** they are **hard to interpret**. Fortunately, we have a very **simple model** with vectors constrained to just **three features**. Therefore, we can **visualize** them using the graphing function I incorporated in the class model.

In [7]:
from plotly import graph_objs as py
# Add your own Plotly Username and api_key
py.sign_in('MarvinBertin', '1d708sl040')

AttributeError: module 'plotly.graph_objs' has no attribute 'sign_in'

In [ ]:
model1.graph_vector_space()

The model was train with very **little data** over a **small number of iterations**, therefore based on how the vectors where **initionalized** the **3D scatter plot** will **differ** a little every time you run it.

Nonetheless, there is a **lot to notice** in the **representation** created by the **neural network**. Firstly, we can observe that the words are definitely **not randomly positioned**. The **word vectors** have been **moved** by the algorithm into so kind of a **structure**. 

For example, the tokens *"the"* and $</s>$ are found at both **opposite extremes** of the plot. The fact that *"the"* was often found at the beginning of a sentence and $</s>$ at the end, may explain this **relationship**.

Furthermore, words like *"skateboarding"*, *"popular"* and *"prince"* are **clustered together**, whereas *"princess"*, *"hates"* and *"but"* form **another cluster**. From different angles, we can observe **additional structures** like *"is popular"* and *"loves attention"*.

These are already pretty **impressive results**, considering how **little data** the model had. It was able to some extent capture some of the **semantics of the text**.

Let's see if we can do even **better!**

## Multi-Word Context Language Model

This model is an extention of the *Continuous Bag-Of_Word Model (CBOW)*. The context is now any **number of words**
before the **target word** we want to predict.

![](image/2neural-network-cbow.png)

To represent this **context of words**, we simply take the **average of the word vectors** in the context. The hidden layer is now the product of this **new vector** and the **input weight matrix**: 

$$h=\frac{1}{C}W_{I}(x_1+x_2+…+x_C)$$

The update for the output weights for the hidden-to-output layer is the same as before:

$${W_{O}}^{T (new)}_j = {W_{O}}^{T (old)}_j - \alpha \cdot e_j \cdot h_j$$

The update for the input weights is almost identical as well. The only difference is that it now needs to be applied to **every word in the context window**.

$${W_{I,c}}^{T (new)}_j = {W_{I,c}}^{T (old)}_j - \frac{1}{C} \cdot \alpha \cdot EH $$


### Python Implementation

The code for this **second model** can be found in file "Word2Vec_nWordContext.py"

The learning algorithm is almost identical to the previous model. However, the challenge is now to find a way to allow the **context window to shrink and expand** as it loops over sentences.

How do you deal with **any size context window** across **any sentence size**?
To illustrate my point, I've created a toy function that helps to visualize how does the **context window moves across a sentence**.

In [ ]:
from context_window import context_window_search

In [ ]:
context_window_search(context_size=2)

As you can see above, when the **window is small** and the **sentence long** this is straight forward. The **window slides** at every iteration. However when you have a **large window** and a **small sentence**, you will run in the problem of giving **less importance** to words at the **beginning and end** of the sentence.

To get around that you need to **dynamically** change the **size of the window** as it slides across the sentence. The function below does this automatically. 

It **expands** gradually to the requested window size and then **contracts** as it reaches the end of the sentence. In this way, every word is passed as input excatly **5 times** for a context size of 5 for example. You can check it for yourself below.

In [ ]:
context_window_search(context_size=5)

This little change allows for **large improvements** in the model. The neural network can now **learning and relate** a word to both its **adjacent** and **distant neighbors**, allowing for **better semantic understanding**.

Let's train this new model on our tiny dataset with this time a **context window of three words**.

In [ ]:
from Word2Vec_nWordContext import Word2Vec_nWordContext
model2 = Word2Vec_nWordContext(sentences, learning_rate = 1.1, context_size = 3, nodes_HL = 3)
W, vocab = model2.train()

In [ ]:
model2.graph_vector_space()

Once again the model **pushed the word vectors** into a **structured vector space**. Some of the same previous strucutres are found here. We can also notice, if viewed the space from **different angles**, that more **subtle relationships** are starting to appear. 

For example, *"princess" - "prince"* **approximate** to *"hates" - "loves"*. In some sense, the model was able to capture **relations** between words that can be mapped into **mathematical operations**.

To be fair, this is not completely obvious from the plot, and some of it may very well just be due to randomness in this case. However as you increase the **data size**, the **search space dimentionality** and **training time**, this model will be able to extract **deeper meaning and semantinc** from a text **efficently**.

## Paragraph Vector Language Model

The Paragraph Vector model is another variation of the previous models. In addition to **mapping every word** into **unique vectors**, the model will also **map the paragraphs or sentences** in the text into a **unique vector**. Each represented by a column in matrix $D$.

<img src="image/3doc2vec.png"  width="500" height="200">

The only real difference in the algorithm is with the **computation of the hidden layer**, $h$. Now both the **sentence vector and word vectors** are **averaged** to predict the next word in a context.

$$h=\frac{1}{C}W_{I}(D_d+x_1+x_2+…+x_C)$$

We can think of the **sentence token** as just another **vocabulary word**. However, this token vector tries to represent the **semantics of the entire sentence**. In other words, it acts as a **memory that remembers what is missing from the current context**. For this reason, this model is also referred as the ** *Distributed Memory Model of Paragraph Vectors (PV-DM)* **.

As the context window travels across a sentence, the **sentence vector is unique**, whereas the **context vector changes** constantly. After being trained, the paragraph vectors can be used as **features** for the paragraph instead of **additional bag-of-words**. These features can then be used directly in other **machine learning models**, such as logistic regression or K-means.

This allows the model to perform **better than a bag-of-n-grams model** because it is able to **generalize the text features** without **suffering from very high-dimensional representations** like in a bag of n-grams model.

### Python Implementation

The implementation of this **third language model** was fairly straight forward, but **concenptually** the **extra paragraph token** added has the potential to greatly boost its **learning capabilities**.

Let's train on our tiny dataset one last time and see the difference this model makes.

In [ ]:
from Doc2Vec_nWordContext import Doc2Vec_nWordContext
model3 = Doc2Vec_nWordContext(sentences, learning_rate = 0.8, context_size = 3, nodes_HL = 3)
W, D, vocab = model3.train()

The training function now also returns the $D$ weight matrix, which is the **learned representation of each five sentences in the text**. 

In [ ]:
D

In [ ]:
model3.graph_vector_space()

The plot of the learned word vectors, now also include the **sentence representations** (S1, S2, etc.). 

The first thing we can notice is that the sentence vectors have been **physically separated** from the word vectors. The model was able to **differentiate** between **individual words** and **entire sections of the text**. 

Furthermore, sentence vectors are grouped into a **strucutre of their own**. **S1 and S3** are together while **S2 and S4** are in another region. The first group of sentences are about **skateboarding in the park**, while the other cluster have sentences with very **similar grammatical structure**. They **compare** what the prince and princess love and hate.

As discussed earlier the Paragraph Vector model takes a **general approach** and is capable of having some sort of **memory**. Therefore it can divide a text into **topics** in an **un-surpervised** way.

This is **powerful** idea which allows to learn a text at **different levels or scales** (words, sentences, paragraphs). In fact, this is similar to **convolution nets** that use **max pooling** to **zoom out** of an image and learning different **feature abstractions** embedded in the image. 

## What's Next?

* Scale up to real datasets
* Learn about NLP libraries like gensim
* Move to more sophisticated models (hierarchical softmax, negative sampling)